In [21]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam



In [2]:
def preprocess_images(dataset_dir, img_size=(48, 48)):
    images = []
    labels = []

    # Define the emotions
    emotions = ["anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]

    # Iterate through the emotion folders
    for i, emotion in enumerate(emotions):
        # Get the path to the emotion folder
        emotion_dir = os.path.join(dataset_dir, emotion)

        # Check if the emotion folder exists
        if os.path.exists(emotion_dir):
            # Iterate through the images in the emotion folder
            for image_name in os.listdir(emotion_dir):
                # Load and preprocess the image
                img_path = os.path.join(emotion_dir, image_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, img_size) / 255.0

                # Append the preprocessed image and label to the lists
                images.append(img)
                labels.append(i)  # Use the index of the emotion as the label

    return np.array(images), np.array(labels)

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [8]:
dataset_dir = "/content/drive/My Drive/images"

In [9]:
images, labels = preprocess_images(dataset_dir)


In [11]:
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (981, 48, 48)
Labels shape: (981,)


In [12]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
print(train_images.shape)
print(test_images.shape)

(784, 48, 48)
(197, 48, 48)


In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax')  # 7 output classes (one for each emotion)
])

In [14]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(train_images, train_labels, validation_split=0.2, epochs=20, batch_size=12)

Epoch 1/20
53/53 [==============================] - 3s 43ms/step - loss: 1.6024 - accuracy: 0.4322 - val_loss: 1.4693 - val_accuracy: 0.5350
Epoch 2/20
53/53 [==============================] - 2s 39ms/step - loss: 0.8765 - accuracy: 0.7241 - val_loss: 0.6728 - val_accuracy: 0.7643
Epoch 3/20
53/53 [==============================] - 2s 39ms/step - loss: 0.4370 - accuracy: 0.8644 - val_loss: 0.5044 - val_accuracy: 0.8344
Epoch 4/20
53/53 [==============================] - 4s 68ms/step - loss: 0.2543 - accuracy: 0.9298 - val_loss: 0.3323 - val_accuracy: 0.8981
Epoch 5/20
53/53 [==============================] - 3s 51ms/step - loss: 0.1281 - accuracy: 0.9713 - val_loss: 0.2394 - val_accuracy: 0.9236
Epoch 6/20
53/53 [==============================] - 2s 40ms/step - loss: 0.0874 - accuracy: 0.9841 - val_loss: 0.2243 - val_accuracy: 0.9299
Epoch 7/20
53/53 [==============================] - 2s 39ms/step - loss: 0.1136 - accuracy: 0.9665 - val_loss: 0.1641 - val_accuracy: 0.9554
Epoch 8/20
53

In [16]:
predictions = model.predict(test_images)

7/7 [==============================] - 0s 19ms/step


In [17]:
print(predictions)

[[1.2153550e-13 3.6275800e-08 7.0996826e-11 ... 7.9533785e-14
  9.9557349e-07 9.9999744e-01]
 [1.8348313e-15 5.6045859e-11 5.2499095e-11 ... 9.9999994e-01
  2.4338185e-15 1.7880398e-14]
 [3.5561892e-04 9.9888480e-01 2.5216696e-07 ... 2.7780450e-06
  9.5873584e-06 5.4612224e-06]
 ...
 [6.8813857e-17 1.0563114e-10 2.2357577e-15 ... 7.6036086e-19
  4.2442085e-09 9.9999952e-01]
 [4.9743033e-11 9.3013950e-06 2.5460384e-08 ... 9.9997234e-01
  6.9648671e-12 5.1279714e-10]
 [6.3839642e-17 5.6864400e-12 3.4376646e-12 ... 1.0000000e+00
  5.8916895e-18 1.8597720e-13]]


In [18]:
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)

[6 4 1 4 0 6 6 4 6 2 4 0 4 4 6 6 0 4 4 6 6 5 4 5 6 3 6 2 2 6 0 0 6 5 2 5 0
 2 6 5 6 6 4 2 0 4 6 2 6 2 2 4 0 6 2 2 6 0 6 1 0 2 6 3 2 5 6 0 4 2 5 2 5 4
 1 0 6 1 4 2 4 2 4 6 6 4 6 6 2 0 2 4 2 2 4 4 2 2 6 4 3 5 5 5 6 6 6 6 0 4 6
 2 0 2 1 0 2 4 3 2 4 4 4 6 2 2 6 4 5 2 4 6 3 2 0 4 4 3 2 4 1 2 4 2 5 0 4 4
 2 2 3 4 6 3 6 6 1 1 4 2 4 6 3 4 6 2 4 3 4 0 0 5 2 6 6 1 4 3 0 1 2 3 2 5 6
 6 5 6 2 0 5 3 4 2 6 4 4]


In [19]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100,"%")

7/7 [==============================] - 0s 19ms/step - loss: 0.1211 - accuracy: 0.9746
Test Loss: 0.12114731967449188
Test Accuracy: 97.46192693710327 %


In [20]:
model.save('model_1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 128)               8